# Whole-genome analysis workflow

In [2]:
# ~2 minutes to install 
#%pip install -U --no-cache-dir scikit-learn scikit-optimize prefect prefect-ray ray plotly openpyxl shap lion_pytorch pytorch_tabnet xgboost neptune pyspark pyarrow dill fastnumbers

In [5]:
from prefect import task, flow
from prefect.task_runners import ConcurrentTaskRunner
from prefect_ray.task_runners import RayTaskRunner
import ray

import pandas as pd
import numpy as np

import logging

!export PREFECT_LOGGING_LEVEL="WARNING"
ray.shutdown()
parallelRunner = ray.init(
  configure_logging=True,
  logging_level=logging.ERROR,
)
parallelRunner

Python version:,3.10.10
Ray version:,2.3.1


In [5]:
from sklearn.ensemble import (
    AdaBoostClassifier,
    RandomForestClassifier,
)
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC, SVC
from xgboost import XGBClassifier
from pytorch_tabnet.tab_model import TabNetClassifier
from lion_pytorch import Lion

from skopt.space import Categorical, Integer, Real

from env import neptune_api_token

RadialBasisSVC = SVC
RadialBasisSVC.__name__ = "RadialBasisSVC"

clearHistory = False

# Edit config in config.py
 
async def remove_all_flows():
  from prefect.client import get_client
  orion_client = get_client()
  flows = await orion_client.read_flows()
  for flow in flows:
    flow_id = flow.id
    print(f"Deleting flow: {flow.name}, {flow_id}")
    await orion_client._client.delete(f"/flows/{flow_id}")
    print(f"Flow with UUID {flow_id} deleted")

if clearHistory: await remove_all_flows()

Deleting flow: bootstrap, 9502aee0-c3fa-4612-a752-19343216f387
Flow with UUID 9502aee0-c3fa-4612-a752-19343216f387 deleted
Deleting flow: classify, 4e5cd54e-d4ae-46d7-85e6-c8f54fcd5719
Flow with UUID 4e5cd54e-d4ae-46d7-85e6-c8f54fcd5719 deleted
Deleting flow: main, 8a8a4b07-fd86-4d09-9f47-5a3e3d541f54
Flow with UUID 8a8a4b07-fd86-4d09-9f47-5a3e3d541f54 deleted
Deleting flow: processInputFiles, 7bd5e911-e0af-4172-9897-874dee374acf
Flow with UUID 7bd5e911-e0af-4172-9897-874dee374acf deleted


In [3]:
from prefect import unmapped
from tqdm import tqdm

from input import processInputFiles

(caseGenotypes,
caseIDs,
holdoutCaseGenotypes,
holdoutCaseIDs,
controlGenotypes,
controlIDs,
holdoutControlGenotypes,
holdoutControlIDs,
clinicalData) = await processInputFiles()

print(f"\nclinical data:\n{clinicalData.head()}")

16:31:30.514 | INFO    | prefect.engine - Created flow run 'mahogany-quetzal' for flow 'processInputFiles'

16:31:30.517 | INFO    | Flow run 'mahogany-quetzal' - View at http://127.0.0.1:4200/flow-runs/flow-run/34327691-543d-471c-b301-1c8fcc690ee8

16:31:30.612 | INFO    | Flow run 'mahogany-quetzal' - Created task run 'load-0' for task 'load'

16:31:30.613 | INFO    | Flow run 'mahogany-quetzal' - Executing 'load-0' immediately...

/home/noot/.pyenv/versions/miniconda3-latest/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


...

16:35:49.203 | INFO    | Flow run 'mahogany-quetzal' - No duplicate columns found!

16:35:49.231 | INFO    | Flow run 'mahogany-quetzal' - Filtered 1421 alleles with frequency below 5.000%

16:35:49.233 | INFO    | Flow run 'mahogany-quetzal' - Kept 61 alleles

16:35:49.329 | INFO    | Flow run 'mahogany-quetzal' - Finished in state Completed()


clinical data:
                       Quote    Data File ID ExternalSubjectId   
ExternalSampleId                                                 
CGND-HDA-05557    CGND_14852  CGND-HDA-05557       NEUUF013XXL  \
CGND-HDA-05556    CGND_14852  CGND-HDA-05556       NEUHM496PGR   
CGND-HDA-05555    CGND_14852  CGND-HDA-05555       NEUPK599KHH   
CGND-HDA-05554    CGND_14852  CGND-HDA-05554       NEUHD589CVP   
CGND-HDA-05553    CGND_14852  CGND-HDA-05553       NEUXX223WT8   

                              Project     Site Sample Collected   
ExternalSampleId                                                  
CGND-HDA-05557    ALS Natural History  Henry Ford Health System  \
CGND-HDA-05556    ALS Natural History  Henry Ford Health System   
CGND-HDA-05555    ALS Natural History  Henry Ford Health System   
CGND-HDA-05554    ALS Natural History  Henry Ford Health System   
CGND-HDA-05553    ALS Natural History  Henry Ford Health System   

                   Site Specimen Collected     Sex 

In [ ]:
len()

In [4]:
len(holdoutControlIDs)

2012

## Evaluate model stack

In [3]:
import neptune
from sklearn.model_selection import StratifiedKFold
from mlStackEntrypoint import classify
from config import config

outerCvIterator = StratifiedKFold(
    n_splits=config["sampling"]["crossValIterations"], shuffle=False
)
innerCvIterator = outerCvIterator
if config["tracking"]["remote"]:
    projectTracker = neptune.init_project(
        project=f'{config["tracking"]["entity"]}/{config["tracking"]["project"]}',
        api_token=config["tracking"]["token"],
    )

results = []
for model, hyperParameterSpace in list(config["model"]["stack"].items()):
    results.append(
        await classify(
            caseGenotypes,
            controlGenotypes,
            holdoutCaseGenotypes,
            holdoutControlGenotypes,
            clinicalData,
            model,
            hyperParameterSpace,
            innerCvIterator,
            outerCvIterator,
        ),
    )

NameError: name 'config' is not defined